In [ ]:
import pandas as pd
import mysql.connector

# 1. MySQL 연결
conn = mysql.connector.connect( #MySQL 서버에 연결 
                                host = "localhost",
                                user = "root",
                                password = "1234",
                                database = "분석실습"
                                )

cursor = conn.cursor()

✅ 커서(cursor) 란?

데이터베이스에서 명령어(SQL)를 보내는 통로

쉽게 말해 MySQL과 대화하는 전화기

```
┌───────────────┐         ┌─────────────┐
│   Python      │ ← 연결 → │   MySQL     │
│  (pandas)     │         │  DB 서버    │
└───────────────┘         └─────────────┘
         ↑
      커서(cursor)
      = SQL 명령 보내고 결과 받는 도구
```

In [6]:
# 2. 파일에서 데이터프레임으로 읽기 (탭 구분자라고 가정)
df_customer = pd.read_csv("DATA/SQL_실습데이터/챕터9_실습데이터/customer.txt", sep="\t", dtype=str, encoding='cp949')
df_customer.shape
df_customer

,mem_no,last_name,first_name,gd,birth_dt,entr_dt,grade,sign_up_ch
0,18466,정,하성,M,1988-03-11,2011-01-01,VVIP,01
1,18798,정,호정,F,1990-02-28,2013-03-09,VVIP,01
2,18434,황,성연,F,1971-04-04,2011-09-12,VIP,03
3,18772,홍,경빈,F,1971-07-06,2012-08-18,VIP,03
4,18326,조,우준,M,1972-05-08,2011-05-26,VIP,01
...,...,...,...,...,...,...,...,...
5364,16332,정,윤제,M,1982-06-03,2012-02-23,BRONZE,01
5365,17987,임,민건,M,1992-04-16,2013-03-18,BRONZE,04
5366,17894,임,정윤,F,1992-04-22,2012-06-03,BRONZE,06
5367,14578,김,로건,M,1992-04-23,2013-10-20,BRONZE,02


In [7]:
# 2. 파일에서 데이터프레임으로 읽기 (탭 구분자라고 가정)
df_sales = pd.read_csv("DATA/SQL_실습데이터/챕터9_실습데이터/sales.txt", sep="\t", dtype=str, encoding='cp949')
df_sales.shape
df_sales

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.1,12680,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680,France


In [ ]:
# 3. customer insert 데이터 넣기

# 3. CUSTOMER INSERT
for row in df_customer.itertuples(index=False, name=None):
    cursor.execute("INSERT INTO CUSTOMER (mem_no, last_name, first_name, gd, birth_dt, entr_dt, grade, sign_up_ch) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)", row)
    
conn.commit()  # 변경사항 저장

cursor.execute("SELECT COUNT(*) FROM CUSTOMER")
print(f"총 {cursor.fetchone()[0]}건 삽입 완료")


총 5369건 삽입 완료


In [14]:
# 4. SALES 데이터전처리
df_sales = df_sales.dropna(subset=["CustomerID"])
df_sales["CustomerID"] = df_sales["CustomerID"].astype(int)

# 날짜 및 숫자 변환
df_sales["Quantitiy"] = df_sales["Quantity"].astype(int)
df_sales["UnitPrice"] = df_sales["UnitPrice"].astype(float)
df_sales["CustomerID"] = df_sales["CustomerID"].astype(int)

# DATETIME 변환
df_sales["InvoiceDate"] = pd.to_datetime(df_sales["InvoiceDate"], errors='coerce')
df_sales["InvoiceDate"] = df_sales["InvoiceDate"].dt.strftime('%Y-%m-%d %H:%M:%S')
df_sales.isna().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
Quantitiy      0
dtype: int64

In [20]:
# 5. SALES INSERT
df_sales = df_sales.astype(object)
rows = [tuple(x) for x in df_sales.to_records(index=False)]
cursor.executemany("""
    INSERT INTO SALES (InvoiceNo, StockCode, Description, Quantitiy, InvoiceDate, UnitPrice, CustomerID, Country)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
""", rows)

conn.commit()

cursor.execute("SELECT COUNT(*) FROM SALES")
print(f"총 {cursor.fetchone()[0]}건 삽입 완료")

DatabaseError: 1265 (01000): Data truncated for column 'Quantitiy' at row 1

In [17]:
df_sales = df_sales.drop(columns=["Quantity"])

In [18]:
print(df_sales.columns)

Index(['InvoiceNo', 'StockCode', 'Description', 'InvoiceDate', 'UnitPrice',
       'CustomerID', 'Country', 'Quantitiy'],
      dtype='object')
